### Loading Libraries

In [ ]:
!pip install tensorflow
!pip install tensorflow-hub
!pip install tensorflow-object-detection-api
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

### Detection Code

In [ ]:
%%writefile Detector.py
import time
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from ultralytics import YOLO  # Import YOLO from Ultralytics

np.random.seed(123)

class Detector:
    def __init__(self):
        self.classesList = []
        self.colorList = []
        self.model = None

    def downloadModel(self, modelName):
        try:
            # Define the model directory
            model_dir = "Model"
            if not os.path.exists(model_dir):
                os.makedirs(model_dir)  # Create the "Model" folder if it doesn't exist
            
            # Construct the path where the model should be saved
            model_path = os.path.join(model_dir, modelName)
            
            if not os.path.exists(model_path):  # Download only if the model isn't already downloaded
                print(f"Downloading YOLO model: {modelName}")
                self.model = YOLO(model_path)  # Load the YOLO model and save it to the specified path
            else:
                print(f"Model {modelName} already exists. Loading from {model_path}")
                self.model = YOLO(model_path)  # Load the model from the saved path
            
            # Set the class names and colors
            self.classesList = self.model.names
            self.colorList = np.random.uniform(low=0, high=255, size=(len(self.classesList), 3))
            print(f"Model {modelName} loaded successfully")
        
        except Exception as e:
            print(f"An error occurred while loading the model: {e}")

    def createBoundingBox(self, image, threshold=0.5):
        try:
            bboxImage = image.copy()
            results = self.model(image)  # Perform inference using YOLOv10

            for result in results:
                boxes = result.boxes
                for box in boxes:
                    # Extract coordinates and other information from YOLOv10's result
                    x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
                    conf = box.conf[0]  # Confidence score
                    class_idx = int(box.cls[0])  # Class index

                    if conf >= threshold:
                        # Get the class label and color dynamically
                        class_label = self.classesList[class_idx] if class_idx < len(self.classesList) else f"ID {class_idx}"
                        color = self.colorList[class_idx]

                        display_text = f"{class_label}: {conf:.2f}"
                        cv2.rectangle(bboxImage, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(bboxImage, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            return bboxImage
        except Exception as e:
            print(f"An error occurred while creating bounding boxes: {e}")
            return None

    def predictImage(self, imagePath, threshold=0.5):
        try:
            img = cv2.imread(imagePath)
            if img is None:
                print(f"Error: Unable to read image from path {imagePath}")
                return
            elif img is not None:
                bboxImage = self.createBoundingBox(img, threshold)
            else:
                print(f"Error: Unable to read image from path {imagePath}")
            if bboxImage is not None:
                cv2.imshow("Detected Image", bboxImage)
                cv2.imwrite("output.jpg", bboxImage)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
            else:
                print("Error: Bounding box image is None.")
        except Exception as e:
            print(f"An error occurred while predicting the image: {e}")
            return None

Writing Detector.py


### Model 

In [5]:
from Detector import Detector
modelName = "yolov10n.pt"  # Replace with the specific YOLOv10 model name
imagePath = "1.png" # Image Path
threshold = 0.002

detector = Detector()
detector.downloadModel(modelName)  # No separate download function is needed
print("Predicting image...")
detector.predictImage(imagePath, threshold)


Loading YOLOv10 model: yolov10n.pt
Model yolov10n.pt loaded successfully
Predicting image...
Error: Unable to read image from path https://isomarsshop.in/cdn/shop/products/1_13d83ebf-0f31-4004-a43a-a164a73c241e_2048x.jpg


### Streamlit App

In [ ]:
%%writefile app.py
import streamlit as st
from Detector import Detector
from PIL import Image
import numpy as np

# Set up the Streamlit app
st.title("Object Detection App")
st.write("Upload an image to perform object detection using YOLOv10")

# Sidebar for selecting model and threshold
modelName = st.sidebar.selectbox(
    "Select YOLOv10 Model",
    ["yolov10n.pt", "yolov10s.pt", "yolov10m.pt", "yolov10b.pt", "yolov10l.pt", "yolov10x.pt"],
    index=0
)

threshold = st.sidebar.slider("Detection Threshold", 0.0, 1.0, 0.5, 0.05)

# Image uploader
uploaded_image = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_image is not None:
    # Convert the uploaded image to an OpenCV format
    image = Image.open(uploaded_image)
    image_np = np.array(image.convert('RGB'))  # Convert to RGB format

    # Initialize the Detector
    detector = Detector()
    detector.downloadModel(modelName)

    # Predict the image
    st.write("Running YOLOv10...")
    result_image = detector.createBoundingBox(image_np, threshold)

    # Display the result image
    st.image(result_image, caption="Detected Objects", use_column_width=True)

else:
    st.write("Please upload an image to get started.")


### Run Web App

In [ ]:
!npm install localtunnel
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl https://loca.lt/mytunnelpassword